# Data cleaning in SQL

General notes:

- To execute SQL commands, prefix with `%sql`. For multiple lines, use `%%sql` and then a newline.

## Environment setup

We'll be using [ipython-sql](https://github.com/catherinedevlin/ipython-sql) to work with SQL directly within a notebook.

In [1]:
%load_ext sql

In [2]:
%sql postgresql://postgres:postgres@db/cleaning

Start from a clean slate:

In [3]:
%sql DROP TABLE IF EXISTS requests;

 * postgresql://postgres:***@db/cleaning
Done.


[]

### Load data

We're taking data from the CSV and putting it in an in-memory [SQLite](https://sqlite.org/index.html) database via [pandas](https://pandas.pydata.org/).

In [4]:
import pandas as pd

requests = pd.read_csv("311_jan_2022.csv", index_col="Unique Key")

In [5]:
%sql --persist requests

 * postgresql://postgres:***@db/cleaning


'Persisted requests'

#### Ensure records were loaded

In [6]:
%sql SELECT * FROM requests LIMIT 3;

 * postgresql://postgres:***@db/cleaning
3 rows affected.


Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
52940375,01/01/2022 12:00:00 AM,01/03/2022 08:39:00 AM,DEP,Department of Environmental Protection,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",None,10036.0,640 8 AVENUE,8 AVENUE,W 41 ST,W 42 ST,None,None,ADDRESS,NEW YORK,None,None,Closed,None,The Department of Environmental Protection determined that this complaint is a duplicate of a previously filed complaint. The original complaint is being addressed.,01/03/2022 08:39:00 AM,05 MANHATTAN,1010137501.0,MANHATTAN,986967.0,214950.0,ONLINE,Unspecified,MANHATTAN,None,None,None,None,None,None,None,40.75666417742652,-73.99019293432467,"(40.75666417742652, -73.99019293432467)"
52934953,01/01/2022 12:00:10 AM,01/01/2022 01:00:11 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11222.0,126 DRIGGS AVENUE,DRIGGS AVENUE,NORTH HENRY STREET,RUSSELL STREET,NORTH HENRY STREET,RUSSELL STREET,ADDRESS,BROOKLYN,DRIGGS AVENUE,None,Closed,None,The Police Department responded to the complaint and with the information available observed no evidence of the violation at that time.,01/01/2022 01:00:15 AM,01 BROOKLYN,None,BROOKLYN,999866.0,202742.0,MOBILE,Unspecified,BROOKLYN,None,None,None,None,None,None,None,40.72314288436064,-73.94366208445774,"(40.72314288436064, -73.94366208445774)"
52933158,01/01/2022 12:00:57 AM,01/01/2022 12:58:22 AM,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,11214.0,45 BAY 38 STREET,BAY 38 STREET,86 STREET,BENSON AVENUE,86 STREET,BENSON AVENUE,ADDRESS,BROOKLYN,BAY 38 STREET,None,Closed,None,The Police Department responded to the complaint and with the information available observed no evidence of the violation at that time.,01/01/2022 12:58:29 AM,11 BROOKLYN,3068667501.0,BROOKLYN,987344.0,156952.0,MOBILE,Unspecified,BROOKLYN,None,None,None,None,None,None,None,40.59747269272421,-73.98885877127528,"(40.59747269272421, -73.98885877127528)"


#### [Display the schema](https://www.postgresql.org/docs/current/app-psql.html#APP-PSQL-META-COMMANDS)

In [7]:
%sql \d requests

 * postgresql://postgres:***@db/cleaning
41 rows affected.


Column,Type,Modifiers
Unique Key,bigint,
Created Date,text,
Closed Date,text,
Agency,text,
Agency Name,text,
Complaint Type,text,
Descriptor,text,
Location Type,text,
Incident Zip,double precision,
Incident Address,text,


## Exploration

### Counts per unique value

In [8]:
%sql SELECT "Agency Name", COUNT("Agency Name") FROM requests GROUP BY "Agency Name";

 * postgresql://postgres:***@db/cleaning
16 rows affected.


Agency Name,count
Department for the Aging,8
Department of Buildings,1233
Department of Consumer Affairs,298
Department of Information Technology and Telecommunications,1
Department of Environmental Protection,2262
Mayorâs Office of Special Enforcement,37
Department of Health and Mental Hygiene,1713
Department of Education,35
New York City Police Department,18254
Department of Homeless Services,501


## Add constraints

### Primary key

In [9]:
%sql ALTER TABLE requests ADD PRIMARY KEY ("Unique Key");

 * postgresql://postgres:***@db/cleaning
Done.


[]

### Convert dates from strings to datetime

In [10]:
%sql SELECT "Created Date" FROM requests LIMIT 3;

 * postgresql://postgres:***@db/cleaning
3 rows affected.


Created Date
01/01/2022 12:00:00 AM
01/01/2022 12:00:10 AM
01/01/2022 12:00:57 AM


In [11]:
%%sql
ALTER TABLE requests
    ALTER COLUMN "Created Date" SET NOT NULL,
    ALTER COLUMN "Created Date" TYPE TIMESTAMP USING to_timestamp("Created Date", 'MM-DD-YYYY HH:MI:SS AM');

 * postgresql://postgres:***@db/cleaning
Done.


[]

In [12]:
%sql SELECT "Created Date" FROM requests LIMIT 3;

 * postgresql://postgres:***@db/cleaning
3 rows affected.


Created Date
2022-01-01 00:00:00
2022-01-01 00:00:10
2022-01-01 00:00:57


#### Your turn: Convert the `Closed Date` to a timestamp

In [15]:
# YOUR CODE HERE

### Ranges

In [17]:
%sql ALTER TABLE requests ADD CHECK ("Latitude" > 40 AND "Latitude" < 41);

 * postgresql://postgres:***@db/cleaning


IntegrityError: (psycopg2.errors.CheckViolation) check constraint "requests_Latitude_check" of relation "requests" is violated by some row

[SQL: ALTER TABLE requests ADD CHECK ("Latitude" > 40 AND "Latitude" < 41);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [18]:
%sql SELECT MIN("Latitude"), MAX("Latitude") FROM requests;

 * postgresql://postgres:***@db/cleaning
1 rows affected.


min,max
2.040074755e-06,40.91216806886205
